In [23]:
import sys
import numpy as np

from pathlib import Path
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mse

sys.path.append("/home/aseliverstov/projects/brain_signals/experiments/src")
from src.dataset import EEG
from src.features import calculate_summary_features
from src.augmentations import augment_timeseries

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/dataset")
intencities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
n_channels = 31

mona_dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels, image_name="Figs for spectra")
cube_dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels, image_name="Cubes for spectra")

mona_dataset.load_to_dataframe(users=[1], signals=["delta"])
cube_dataset.load_to_dataframe(users=[1], signals=["delta"])

# Model

In [22]:
X_train, y_train = mona_dataset.X, mona_dataset.y
X_test, y_test = cube_dataset.X, cube_dataset.y

X_train_stacked = X_train.reshape(len(intencities), n_channels, -1)
X_test_stacked = X_test.reshape(len(intencities), n_channels, -1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
def calculate_correlations(stacked_arr, n_intencities = 10, n_channels = 31):
    correlations = []

    for i in range(n_intencities):
        cor_matrix = np.corrcoef(stacked_arr[i]).ravel()
        cor_matrix = cor_matrix[:len(cor_matrix) // 2]
        for _ in range(n_channels):
            correlations.append(cor_matrix)

    correlations = np.vstack(correlations)
    return correlations

X_train_corr = calculate_correlations(X_train_stacked)
X_test_corr = calculate_correlations(X_test_stacked)

X_train_corr.shape

(310, 480)

In [29]:
X_train_stat = calculate_summary_features(X_train, n_windows=50)
X_test_stat = calculate_summary_features(X_test, n_windows=50)

X_train_stat.shape

(310, 300)

In [30]:
X_train = np.hstack([X_train_corr, X_train_stat])
X_test = np.hstack([X_test_corr, X_test_stat])

X_train.shape

(310, 780)

In [34]:
model = Sequential()
model.add(Dense(64, input_shape= (X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=mse,
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=40, batch_size=32, verbose=1)

Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 11.1487 - val_loss: 0.2825
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.3793 - val_loss: 0.2234
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2391 - val_loss: 0.1474
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1331 - val_loss: 0.1016
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1074 - val_loss: 0.0860
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0994 - val_loss: 0.0831
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0864 - val_loss: 0.0828
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0915 - val_loss: 0.0831
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0856 - val_loss: 0.0833
Epoch 10/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0840 - val_loss: 0.0830
Epoch 11/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0828 - val_loss: 0.0830
Epoch 12/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.

In [55]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_93 (Dense)                │ (None, 64)             │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_72 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_73 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_95 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_74 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_96 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_75 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_97 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,653 (272.09 KB)

 Trainable params: 23,217 (90.69 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 46,436 (181.39 KB)